In [1]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
import time
import talib as ta

import warnings
warnings.filterwarnings("ignore")

### 登入帳號

In [2]:
server = "Yuanta-Demo"
login = 9887788
password = "tery7014"

mt5.initialize(path=r"C:\Program Files\MetaTrader 5\terminal64.exe")
mt5.login(login=int(login), server=server, password=password)

mt5.last_error()

(1, 'Success')

### 下單Functions

In [3]:
def get_data(symbol, timeframe):
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)
    df = pd.DataFrame(bars).dropna()
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

In [4]:
def market_order(symbol, volume, order_type, sl, tp, magic, **kwargs):
    if order_type == 'buy':
        price = mt5.symbol_info_tick(symbol).ask

        if sl == None and tp == None:
            sl = price*0.5
            tp = price*1.5
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": mt5.ORDER_TYPE_BUY_LIMIT,
            "price": price,
            "sl": sl, 
            "tp": tp,
            "deviation": 20,
            "magic": magic,
            "comment": "python market order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
            }
    elif order_type == 'sell':
        price = mt5.symbol_info_tick(symbol).bid

        if sl == None and tp == None:
            sl = price*1.5
            tp = price*0.5
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": mt5.ORDER_TYPE_SELL_LIMIT,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 20,
            "magic": magic,
            "comment": "python market order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
            }
    order_result = mt5.order_send(request)

    return order_result.comment

In [5]:
def closing_position(symbol, magic):
        if (magic == None) & (symbol == None):
                position = mt5.positions_get()
                for i in position:
                
                        close_request={
                                "action": mt5.TRADE_ACTION_DEAL,
                                "symbol": i.symbol,
                                "volume": i.volume,
                                "type": mt5.ORDER_TYPE_BUY if i.type == 1 else mt5.ORDER_TYPE_SELL,
                                "position": i.ticket,
                                "price": mt5.symbol_info_tick(i.symbol).ask if i.type == 1 else mt5.symbol_info_tick(i.symbol).bid,
                                "deviation": 20,
                                "magic": i.magic,
                                "comment": "python script close",
                                "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
                                "type_filling": mt5.ORDER_FILLING_RETURN,
                        }
                        # send a close request
                        order_result = mt5.order_send(close_request)

                return order_result.comment

        elif (magic != None) & (symbol != None):
                position = mt5.positions_get()
                for i in position:
                        if (i.magic == magic) & (i.symbol == symbol):
                                close_request={
                                        "action": mt5.TRADE_ACTION_DEAL,
                                        "symbol": i.symbol,
                                        "volume": i.volume,
                                        "type": mt5.ORDER_TYPE_BUY if i.type == 1 else mt5.ORDER_TYPE_SELL,
                                        "position": i.ticket,
                                        "price": mt5.symbol_info_tick(i.symbol).ask if i.type == 1 else mt5.symbol_info_tick(i.symbol).bid,
                                        "deviation": 20,
                                        "magic": i.magic,
                                        "comment": "python script close",
                                        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
                                        "type_filling": mt5.ORDER_FILLING_RETURN,
                                }
                                # send a close request
                                order_result = mt5.order_send(close_request)

                return order_result.comment

In [6]:
def trailingSL(exposure, timeframe):

    if exposure is not None:
        for i in range(len(exposure)):
            
            symbol = exposure['symbol'][i]
            volume = exposure['volume'][i]
            sl = exposure['sl'][i]
            tp = exposure['tp'][i]
            price_open = exposure['price_open'][i]
            ticket = exposure['ticket'][i]

            if exposure['type'][i] == 0:
                type = mt5.ORDER_TYPE_BUY
            elif exposure['type'][i] == 1:
                type = mt5.ORDER_TYPE_SELL

            bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)  #最近前五根k棒的最高價or最低價停損
            df = pd.DataFrame(bars).dropna()
            df['sar'] = ta.SAR(df['high'], df['low'])

            long_sl = df.iloc[-2]['sar']
            short_sl = df.iloc[-2]['sar']

            if (long_sl > sl) & (type == mt5.ORDER_TYPE_BUY):  #long單
                sl = long_sl
                tp = tp 
            
            elif (short_sl < sl) & (type == mt5.ORDER_TYPE_SELL):  #short單
                sl = short_sl
                tp = tp 

            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "symbol": symbol,
                "position": int(ticket),
                "sl": sl,
                "tp": tp,
                "magic": 1,
            }


            order_result = mt5.order_send(request)

        return order_result.comment

In [7]:
def get_exposure(symbol):
    positions = mt5.positions_get(symbol=symbol)
    if positions:
        pos_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())
        pos_df['time'] = pd.to_datetime(pos_df['time'], unit='s')
        exposure = pos_df[['ticket', 'magic', 'symbol', 'type', 'price_open', 'volume', 'sl', 'tp', 'profit']]

        return exposure

In [8]:
def crossover(df1, df2):
    if ((df1.iloc[-2] > df2.iloc[-2]) & (df1.iloc[-3] < df2.iloc[-3])) | ((df1.iloc[-2] < df2.iloc[-2]) & (df1.iloc[-3] > df2.iloc[-3])):
        return True
    else:
        return False

In [69]:
def emerge_close_position(symbol):
    exposure = get_exposure(symbol)
    #if exposure is not None:
    for i in range(len(exposure)):

        if exposure.iloc[i]['magic'] == 1:
            magic1 = 0

            df1 = get_data(symbol, mt5.TIMEFRAME_M15)
            df1['macd'], df1['signal'], df1['hist'] = ta.MACD(df1.close)

            if (exposure.iloc[i]['type'] == 0) & ((df1.iloc[-2]['hist'] < 0) & (df1.iloc[-3]['hist'] > 0)) & (datetime.now().minute%15 == 0):
                closing_position(symbol=symbol, magic=1)
            
            elif (exposure.iloc[i]['type'] == 1) & ((df1.iloc[-2]['hist'] > 0) & (df1.iloc[-3]['hist'] < 0)) & (datetime.now().minute%15 == 0):
                closing_position(symbol=symbol, magic=1)

            else:
                print('1不須緊急出場')


        if exposure.iloc[i]['magic'] == 2:
            magic2 = 0
            #return magic2

        if exposure.iloc[i]['magic'] == 3:
            magic3 = 0
            
            df3 = get_data(symbol, mt5.TIMEFRAME_H1)
            df3['rsi25'] = ta.RSI(df3.close, 25)
            df3['rsi100'] = ta.RSI(df3.close, 100)

            if (exposure.iloc[i]['type'] == 0) & ((df3.iloc[-2]['rsi25'] < df3.iloc[-2]['rsi100']) & (df3.iloc[-3]['rsi25'] > df3.iloc[-3]['rsi100'])) & (datetime.now().minute%60 == 0):
                closing_position(symbol=symbol, magic=3)
            
            elif (exposure.iloc[i]['type'] == 1) & ((df3.iloc[-2]['rsi25'] > df3.iloc[-2]['rsi100']) & (df3.iloc[-3]['rsi25'] < df3.iloc[-3]['rsi100'])) & (datetime.now().minute%60 == 0):
                closing_position(symbol=symbol, magic=3)

            else:
                print('3不須緊急出場')



### 策略撰寫

In [9]:
def signal(symbol, timeframe):
    magic = 1
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 2000)
    df = pd.DataFrame(bars).dropna()
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
    df['ema'] = ta.EMA(df['close'], 200)
    df['macd'], df['dif'], df['hist'] = ta.MACD(df['close'], 12, 26, 9)
    df['sar'] = ta.SAR(df['high'], df['low'], 0.02, 0.2)

    order = None

    con1 = df.iloc[-2]['close'] > df.iloc[-2]['ema']
    con2 = df.iloc[-2]['close'] > df.iloc[-2]['sar']
    con3 = df.iloc[-2]['close'] > df.iloc[-2]['open']
    con4 = (df.iloc[-2]['hist'] >= 0) & (df.iloc[-3]['hist'] < 0)

    con5 = df.iloc[-2]['close'] < df.iloc[-2]['ema']
    con6 = df.iloc[-2]['close'] < df.iloc[-2]['sar']
    con7 = df.iloc[-2]['close'] < df.iloc[-2]['open']
    con8 = (df.iloc[-2]['hist'] <= 0) & (df.iloc[-3]['hist'] > 0)
    
    if (con1 & con2 & con4):
        order = 'buy'
        sl = df.iloc[-2]['sar']
        tp = (2*df.iloc[-1]['open'] - df.iloc[-2]['sar'])
        return df.iloc[-1]['open'], order, sl, tp, magic
    
    elif (con5 & con6 & con8):
        order = 'sell'
        sl = df.iloc[-2]['sar']
        tp = abs(df.iloc[-2]['sar'] - 2*df.iloc[-1]['open'])
        return df.iloc[-1]['open'], order, sl, tp, magic

    else:
        sl = 0
        tp = 0
        magic = None
        return df.iloc[-1]['close'], order, sl, tp, magic

In [10]:
def signal2(symbol, timeframe):
    magic = 2
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)
    df = pd.DataFrame(bars).dropna()
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
    df['ema'] = ta.EMA(df['close'], 10)
    df['sma'] = ta.SMA(df['close'], 10)
    df['rsi'] = ta.RSI(df['close'], 14)

    order = None

    con1 = df.iloc[-2]['ema'] > df.iloc[-2]['sma']
    con2 = (df.iloc[-2]['low'] > df.iloc[-2]['ema']) & (df.iloc[-2]['low'] > df.iloc[-2]['sma'])
    con3 = (df.iloc[-2]['rsi'] > 50) & (df.iloc[-2]['rsi'] < 70)
    con4 = df.iloc[-2]['close'] > df.iloc[-2]['open']

    con5 = df.iloc[-2]['ema'] < df.iloc[-2]['sma']
    con6 = (df.iloc[-2]['high'] < df.iloc[-2]['ema']) & (df.iloc[-2]['high'] < df.iloc[-2]['sma'])
    con7 = (df.iloc[-2]['rsi'] > 30) & (df.iloc[-2]['rsi'] < 50)
    con8 = df.iloc[-2]['close'] < df.iloc[-2]['open']
    
    if (con1 & con2 & con3 & con4):
        order = 'buy'
        sl = df.iloc[-2]['ema']
        tp = (2*df.iloc[-2]['open'] - df.iloc[-2]['ema'])
        
        return df.iloc[-1]['open'], order, sl, tp, magic
    
    elif (con5 & con6 & con7 & con8):
        order = 'sell'
        sl = df.iloc[-2]['ema']
        tp = (abs(2*df.iloc[-2]['open'] - df.iloc[-2]['ema']))

        return df.iloc[-1]['open'], order, sl, tp, magic

    else:
        sl = 0
        tp = 0
        magic = None
        return df.iloc[-1]['close'], order, sl, tp, magic
        
    
signal2('EURUSD.x', mt5.TIMEFRAME_M5)

(1.06992, None, 0, 0, None)

In [11]:
def super_trend(high, low, close, atr, multiplier):
#计算超级趋势指标
    hl2 = (high + low) / 2
    final_upperband = hl2 + (multiplier * atr)
    final_lowerband = hl2 - (multiplier * atr)

    st = [1] * len(close) # True代表上行趋势，False代表下行趋势

    for i in range(1, len(close)):
        curr = i 
        prev = i - 1

        # 如果当前收盘价上传上轨
        if close[curr] > final_upperband[prev]:
            st[curr] = 1
        # 当前收盘价下穿下轨
        elif close[curr] < final_lowerband[prev]:
            st[curr] = 0
        # 否则趋势延续
        else:
            st[curr] = st[prev]
            if st[curr] and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if not st[curr] and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # 根据趋势方向分别移除相应的上下轨
        if st[curr]:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
        
        
    return st

def signal3(symbol, timeframe):
    magic = 3
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)
    df = pd.DataFrame(bars).dropna()
    df['time'] = pd.to_datetime(df['time'], unit='s')

    df['rsi25'] = ta.RSI(df.close, 25)
    df['rsi100'] = ta.RSI(df.close, 100)

    st = super_trend(df.high, df.low, df.close, ta.ATR(df.high, df.low, df.close, 22), 3)

    order = None

    con1 = (st[-2] == 1)
    con2 = (df.iloc[-3]['rsi25'] < df.iloc[-3]['rsi100']) & (df.iloc[-2]['rsi25'] > df.iloc[-2]['rsi100'])

    con3 = (st[-2] == 0)
    con4 = (df.iloc[-3]['rsi25'] > df.iloc[-2]['rsi100']) & (df.iloc[-1]['rsi25'] < df.iloc[-2]['rsi100'])

    if (con1 & con2):
        order = 'buy'

        return df.iloc[-1]['open'], order, magic
    
    elif (con3 & con4):
        order = 'sell'

        return df.iloc[-1]['open'], order, magic
    
    else:
        magic = None
        return df.iloc[-1]['open'], order, magic

In [12]:
df = get_data('SPX500', mt5.TIMEFRAME_M15)

df['time'] = pd.to_datetime(df['time'], unit='s')

df['ema'] = ta.EMA(df['close'], 200)
df['macd'], df['dif'], df['hist'] = ta.MACD(df['close'], 12, 26, 9)
df['sar'] = ta.SAR(df['high'], df['low'], 0.02, 0.2)

df.tail(5)

,time,open,high,low,close,tick_volume,spread,real_volume,ema,macd,dif,hist,sar
995,2023-02-16 16:15:00,4156.1,4158.2,4152.5,4152.7,741,2,0,4134.384050,0.544088,1.014523,-0.470435,4151.360170
996,2023-02-16 16:30:00,4152.8,4153.0,4150.2,4151.2,646,2,0,4134.551373,0.245805,0.860780,-0.614975,4159.700000
997,2023-02-16 16:45:00,4151.0,4152.2,4149.7,4149.9,427,2,0,4134.704095,-0.094398,0.669744,-0.764142,4159.510000
998,2023-02-16 17:00:00,4149.7,4155.0,4148.0,4155.0,644,2,0,4134.906045,0.046975,0.545190,-0.498215,4159.117600
999,2023-02-16 17:15:00,4155.0,4155.7,4152.4,4152.5,308,2,0,4135.081109,-0.042229,0.427707,-0.469935,4158.450544


### 自動交易

In [13]:
while True:
    symbols_list = {

        "1": ["US30", 5.0],
        "2": ["SPX500", 5.0],
        "3": ["NK225", 50.0],
        "4": ["NAS100", 5.0],
        "5": ["EURUSD.x", 2.0]
        #"7": ["USDJPY.x", 0.5, mt5.TIMEFRAME_M5],

    }

    print("現在時間：",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # if datetime.now().weekday() == 5 and datetime.now().strftime("%H:%M") == '05:55':
    #     closing_position(None, None)
    #     print('放假了~全部出清~')
    #     break

    for asset in symbols_list.keys():
        # Initialize the inputs
        symbol = symbols_list[asset][0]
        volume = symbols_list[asset][1]
        #timeframe = symbols_list[asset][2]
        exposure = get_exposure(symbol)

        price1, order1, sl1, tp1, magic1 = signal(symbol=symbol, timeframe=mt5.TIMEFRAME_M15)
        price2, order2, sl2, tp2, magic2 = signal2(symbol=symbol, timeframe=mt5.TIMEFRAME_M5)
        price3, order3, magic3 = signal3(symbol=symbol, timeframe=mt5.TIMEFRAME_H1)
        magic2 = None

        if exposure is not None:
            for i in range(len(exposure)):
                if exposure.iloc[i]['magic'] == 1:
                    magic1 = 0
                elif exposure.iloc[i]['magic'] == 2:
                    magic2 = 0
                elif exposure.iloc[i]['magic'] == 3:
                    magic3 = 0
                
        emerge_close_position(symbol)  #緊急出場確認


        if (order1 in ['sell', 'buy']) & (magic1 == 1) & (datetime.now().minute%15 == 0):
            
            print(f'建倉種類：{order1} 商品種類：{symbol} 倉位：{volume} 執行狀況：{market_order(symbol, volume, order1, sl1, tp1, magic1)}')

        if (order2 in ['sell', 'buy']) & (magic2 == 2) & (datetime.now().minute%5 == 0):
            
           print(f'建倉種類：{order2} 商品種類：{symbol} 倉位：{volume} 執行狀況：{market_order(symbol, volume=0.1, order_type=order2, price=price2, sl=sl2, tp=tp2, magic=magic2)}')

        if (order1 in ['sell', 'buy']) & (magic1 == 3) & (datetime.now().minute%60 == 0):
            
            print(f'建倉種類：{order1} 商品種類：{symbol} 倉位：{volume} 執行狀況：{market_order(symbol, volume, order_type=order3, sl=None, tp=None, magic=magic3)}')

        print(f'商品種類：{symbol}，目前價位：{price1}，訊號判斷{magic1, magic2, magic3}')


        print("現有部位：\n", get_exposure(symbol))
        print('============================================================')
    

    time.sleep(5)

現在時間： 2023-02-16 17:24:40
商品種類：US30，目前價位：34143.0，訊號判斷(None, None, None)
現有部位：
 None
商品種類：SPX500，目前價位：4152.5，訊號判斷(None, None, None)
現有部位：
 None
商品種類：NK225，目前價位：27667.0，訊號判斷(None, None, None)
現有部位：
 None
商品種類：NAS100，目前價位：12718.5，訊號判斷(None, None, None)
現有部位：
 None
第一策略不須緊急出場
商品種類：EURUSD.x，目前價位：1.06992，訊號判斷(0, None, None)
現有部位：
     ticket  magic    symbol  type  price_open  volume       sl       tp  \
0  4605617      1  EURUSD.x     0     1.07177     2.0  1.06967  1.07387   

   profit  
0  -370.0  
現在時間： 2023-02-16 17:24:45
商品種類：US30，目前價位：34144.0，訊號判斷(None, None, None)
現有部位：
 None
商品種類：SPX500，目前價位：4152.5，訊號判斷(None, None, None)
現有部位：
 None
商品種類：NK225，目前價位：27665.0，訊號判斷(None, None, None)
現有部位：
 None
商品種類：NAS100，目前價位：12718.9，訊號判斷(None, None, None)
現有部位：
 None
第一策略不須緊急出場
商品種類：EURUSD.x，目前價位：1.06991，訊號判斷(0, None, None)
現有部位：
     ticket  magic    symbol  type  price_open  volume       sl       tp  \
0  4605617      1  EURUSD.x     0     1.07177     2.0  1.06967  1.07387   

   profit  
0  -372.

KeyboardInterrupt: 